In [1]:
pip install xlrd>=2.0.1

Note: you may need to restart the kernel to use updated packages.


In [9]:
#новый код олько для города    

import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils import get_column_letter


INPUT_FILE  = "file_path"
OUTPUT_FILE = "Bishkek2fix_errors.xlsx"



def to_num(s):
    return pd.to_numeric(s, errors="coerce")

def norm(name: str) -> str:
    """Нормализует имя колонки (для сопоставления)"""
    return (name or "").lower().replace(" ", "").replace(".", "").replace("(", "").replace(")", "").replace("_", "").replace("-", "")

def find_column(df, pattern):
    for c in df.columns:
        if pattern in norm(c):
            return c
    return None


def process_df(df, filial_name):
    """Основная логика обработки одного филиала"""
    df = df.copy()
    df.columns = [norm(c) for c in df.columns]
    df.insert(0, "№", range(1, len(df) + 1))

   
    patterns = {
        "адрес": "адрес",
        "суммасделки": "суммасдел",
        "плземучквм": "плзем",
        "плстроения": "плстро",
        "этаж": "этаж",
        "колвоэтажейвздании": "колвоэтаж",
        "формаиспользования": "формаисп"
    }

    renames = {}
    for new_col, pattern in patterns.items():
        found = find_column(df, pattern)
        if found:
            renames[found] = new_col
        else:
            print(f"⚠️ [{filial_name}] Не найдена колонка, похожая на: {pattern}")
    df.rename(columns=renames, inplace=True)

    # ---------- создаём колонку "Сумма за кв.м" ----------
    if "суммасделки" in df.columns and "плземучквм" in df.columns:
        sum_deal = to_num(df["суммасделки"])
        area = to_num(df["плземучквм"])
        result = sum_deal / area
        error_mask = result.isna() | (area == 0)
        df["суммазаквм"] = result
        df.loc[error_mask, "суммазаквм"] = "ошибка"
    else:
        print(f"⚠️ [{filial_name}] Не удалось создать колонку 'Сумма за кв.м'")

    # ---------- условия ----------
    conditions = {
    1: {
        "desc": "Колонка 'Адрес' — 0 или пустая строка",
        "mask": lambda d: (
            d["адрес"].isna() |
            (d["адрес"].astype(str).str.strip().isin(["", "0"]))
        )
    },

    2: {
        "desc": "Форма ≠ земельный участок/земельный участок с _ / незавершённый объект → Сумма сделки ≤ 412800, 0, отрицательные, пустые",
        "mask": lambda d: (
            ~d["формаиспользования"].astype(str).str.contains(
                "земельный участок|земельный участок с|незаверш", case=False, na=False
            )
            &
            (
                d["суммасделки"].isna()
                | (to_num(d["суммасделки"]) <= 0)
                | (to_num(d["суммасделки"]) < 412800)
            )
        )
    },

    3: {
        "desc": "Форма ≠ земельный участок/земельный участок с _ / незавершённый объект → Пл.строения 0, пустая, <12, >1 000 000",
        "mask": lambda d: (
            ~d["формаиспользования"].astype(str).str.contains(
                "земельный участок|земельный участок с|незаверш", case=False, na=False
            )
            &
            (
                d["плстроения"].isna()
                | (to_num(d["плстроения"]) <= 0)
                | (to_num(d["плстроения"]) < 12)
                | (to_num(d["плстроения"]) > 1_000_000)
            )
        )
    },

    4: {
        "desc": "Форма ≠ земельный участок/земельный участок с _ / незавершённый объект → Этаж 0, пустая, дробное, >23",
        "mask": lambda d: (
            ~d["формаиспользования"].astype(str).str.contains(
                "земельный участок|земельный участок с|незаверш", case=False, na=False
            )
            &
            (
                d["этаж"].isna()
                | (to_num(d["этаж"]) == 0)
                | (to_num(d["этаж"]) > 23)
                | ((to_num(d["этаж"]) % 1) != 0)
            )
        )
    },

    5: {
        "desc": "Форма ≠ земельный участок/земельный участок с _ / незавершённый объект → Кол-во этажей 0, пустая, дробное, >23",
        "mask": lambda d: (
            ~d["формаиспользования"].astype(str).str.contains(
                "земельный участок|земельный участок с|незаверш", case=False, na=False
            )
            &
            (
                d["колвоэтажейвздании"].isna()
                | (to_num(d["колвоэтажейвздании"]) == 0)
                | (to_num(d["колвоэтажейвздании"]) > 23)
                | ((to_num(d["колвоэтажейвздании"]) % 1) != 0)
            )
        )
    },

    6: {
        "desc": "Форма = земельный участок/Пашня/Пашня орошаемая/Пашня богарная → Сумма за кв.м 0, пустая, <172",
        "mask": lambda d: (
            d["формаиспользования"].astype(str).str.contains(
                "земельный участок|пашня|пашня орошаемая|пашня богарная", case=False, na=False
            )
            &
            (
                d["суммазаквм"].isna()
                | (to_num(d["суммазаквм"]) == 0)
                | (to_num(d["суммазаквм"]) < 172)
            )
        )
    },

    7: {
        "desc": "Форма = земельный участок/Пашня/Пашня орошаемая/Пашня богарная → Пл.зем.уч.(кв.м) 0, пустая, <100",
        "mask": lambda d: (
            d["формаиспользования"].astype(str).str.contains(
                "земельный участок|пашня|пашня орошаемая|пашня богарная", case=False, na=False
            )
            &
            (
                d["плземучквм"].isna()
                | (to_num(d["плземучквм"]) <= 0)
                | (to_num(d["плземучквм"]) < 100)
            )
        )
    },
}


    # ---------- каскад ----------
    summary, error_blocks = [], []
    cur = df.copy()
    total_initial = len(cur)

    for cond_no, spec in conditions.items():
        mask = spec["mask"](cur)
        err_idx = cur.index[mask]
        errors_df = cur.loc[err_idx].copy()

        total_rows = len(cur)
        error_count = len(errors_df)
        error_percent = (error_count / total_rows * 100) if total_rows else 0

        summary.append({
            "Условие": cond_no,
            "Описание": spec["desc"],
            "Ошибок": int(error_count),
            "Всего строк": int(total_rows),
            "% ошибок": round(error_percent, 2)
        })

        if error_count:
            errors_df["Условие"] = cond_no
            error_blocks.append(errors_df)

        cur = cur.drop(index=err_idx).copy()

    final_rows = len(cur)
    final_percent = round(final_rows / total_initial * 100, 2)
    summary.append({
        "Условие": "✅ ИТОГ",
        "Описание": "Чистых строк после всех проверок",
        "Ошибок": "",
        "Всего строк": final_rows,
        "% ошибок": f"Осталось {final_percent}%"
    })

    return summary, error_blocks, cur


# ---------- обработка всех листов ----------
excel = pd.ExcelFile(INPUT_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    for sheet_name in excel.sheet_names:
        print(f"🔍 Обработка листа: {sheet_name}")
        df = excel.parse(sheet_name)

        filial_col = find_column(df, "филиал")
        if filial_col and df[filial_col].notna().any():
            filial_name = str(df[filial_col].dropna().iloc[0])[:31]
        else:
            filial_name = sheet_name[:31]

        summary, errors, clean = process_df(df, filial_name)

        # Пишем summary
        summary_df = pd.DataFrame(summary)
        summary_df.to_excel(writer, sheet_name=filial_name, index=False)
        start_row = len(summary_df) + 3

        # Пишем ошибки
        for block in errors:
            cond_no = int(block["Условие"].iloc[0])
            block.drop(columns=["Условие"]).to_excel(writer, sheet_name=filial_name, index=False, startrow=start_row)
            start_row += len(block) + 3

       
        clean.to_excel(writer, sheet_name=filial_name, index=False, startrow=start_row + 1)

print(f"✅ Готово! Все филиалы сохранены в {OUTPUT_FILE}")




🔍 Обработка листа: Sheet1


C:\Users\User\AppData\Local\Temp\ipykernel_21976\2101446031.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ошибка' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[error_mask, "суммазаквм"] = "ошибка"


✅ Готово! Все филиалы сохранены в Bishkek2fix_errors.xlsx


In [ ]:
#новый код для всех остальных регионов 

import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils import get_column_letter


INPUT_FILE  = "Regions_path" 
OUTPUT_FILE = "region fix_errors.xlsx"



def to_num(s):
    return pd.to_numeric(s, errors="coerce")

def norm(name: str) -> str:
    """Нормализует имя колонки (для сопоставления)"""
    return (name or "").lower().replace(" ", "").replace(".", "").replace("(", "").replace(")", "").replace("_", "").replace("-", "")

def find_column(df, pattern):
    for c in df.columns:
        if pattern in norm(c):
            return c
    return None


def process_df(df, filial_name):
    """Основная логика обработки одного филиала"""
    df = df.copy()
    df.columns = [norm(c) for c in df.columns]
    df.insert(0, "№", range(1, len(df) + 1))

    # ---------- гибкое сопоставление колонок ----------
    patterns = {
        "адрес": "адрес",
        "суммасделки": "суммасдел",
        "плземучквм": "плзем",
        "плстроения": "плстро",
        "этаж": "этаж",
        "колвоэтажейвздании": "колвоэтаж",
        "виднедвижимости": "виднедв",
        "формаиспользования": "формаисп"
    }

    renames = {}
    for new_col, pattern in patterns.items():
        found = find_column(df, pattern)
        if found:
            renames[found] = new_col
        else:
            print(f"⚠️ [{filial_name}] Не найдена колонка, похожая на: {pattern}")
    df.rename(columns=renames, inplace=True)

    # ---------- создаём колонку "Сумма за кв.м" ----------
    if "суммасделки" in df.columns and "плземучквм" in df.columns:
        sum_deal = to_num(df["суммасделки"])
        area = to_num(df["плземучквм"])
        result = sum_deal / area
        error_mask = result.isna() | (area == 0)
        df["суммазаквм"] = result
        df.loc[error_mask, "суммазаквм"] = "ошибка"
    else:
        print(f"⚠️ [{filial_name}] Не удалось создать колонку 'Сумма за кв.м'")

    # ---------- условия ----------
    conditions = {
        1: {
            "desc": "Колонка 'Адрес' — 0 или пустая строка",
            "mask": lambda d: (
                d["адрес"].isna() |
                (d["адрес"].astype(str).str.strip().isin(["", "0"]))
            )
        },
        2: {
            "desc": "Форма ≠ Земельные/незавершённые, Вид ≠ Зем.участок без строения → Сумма сделки ≤ 412800, 0, отрицательные, пустые",
            "mask": lambda d: (
                ~d["формаиспользования"].astype(str).str.contains("земельный участок|незаверш", case=False, na=False)
                & ~d["виднедвижимости"].astype(str).str.contains("зем.участок без строения", case=False, na=False)
                &
                (
                    d["суммасделки"].isna()
                    | (to_num(d["суммасделки"]) <= 0)
                    | (to_num(d["суммасделки"]) < 412800)
                )
            )
        },
        3: {
            "desc": "Форма ≠ Земельные/незавершённые → Пл.строения 0, пустая, <12, >1 000 000",
            "mask": lambda d: (
                ~d["формаиспользования"].astype(str).str.contains("земельный участок|незаверш", case=False, na=False)
                & ~d["виднедвижимости"].astype(str).str.contains("зем.участок без строения", case=False, na=False)
                &
                (
                    d["плстроения"].isna()
                    | (to_num(d["плстроения"]) <= 0)
                    | (to_num(d["плстроения"]) < 12)
                    | (to_num(d["плстроения"]) > 1_000_000)
                )
            )
        },
        4: {
            "desc": "Форма ≠ Земельные/незавершённые → Этаж 0, пустая, дробное, >23",
            "mask": lambda d: (
                ~d["формаиспользования"].astype(str).str.contains("земельный участок|незаверш", case=False, na=False)
                & ~d["виднедвижимости"].astype(str).str.contains("зем.участок без строения", case=False, na=False)
                &
                (
                    d["этаж"].isna()
                    | (to_num(d["этаж"]) == 0)
                    | (to_num(d["этаж"]) > 23)
                    | ((to_num(d["этаж"]) % 1) != 0)
                )
            )
        },
        5: {
            "desc": "Форма ≠ Земельные/незавершённые → Кол-во этажей 0, пустая, дробное, >23",
            "mask": lambda d: (
                ~d["формаиспользования"].astype(str).str.contains("земельный участок|незаверш", case=False, na=False)
                & ~d["виднедвижимости"].astype(str).str.contains("зем.участок без строения", case=False, na=False)
                &
                (
                    d["колвоэтажейвздании"].isna()
                    | (to_num(d["колвоэтажейвздании"]) == 0)
                    | (to_num(d["колвоэтажейвздании"]) > 23)
                    | ((to_num(d["колвоэтажейвздании"]) % 1) != 0)
                )
            )
        },
        6: {
            "desc": "Форма = Земельный участок/Пашня, Вид = Зем.участок без строения → Сумма за кв.м 0, пустая, <172",
            "mask": lambda d: (
                d["формаиспользования"].astype(str).str.contains("земельный участок|пашня", case=False, na=False)
                & d["виднедвижимости"].astype(str).str.contains("зем.участок без строения", case=False, na=False)
                &
                (
                    d["суммазаквм"].isna()
                    | (to_num(d["суммазаквм"]) == 0)
                    | (to_num(d["суммазаквм"]) < 172)
                )
            )
        },
        7: {
            "desc": "Форма = Земельный участок/Пашня, Вид = Зем.участок без строения → Пл.зем.уч.(кв.м) 0, пустая, <100",
            "mask": lambda d: (
                d["формаиспользования"].astype(str).str.contains("земельный участок|пашня", case=False, na=False)
                & d["виднедвижимости"].astype(str).str.contains("зем.участок без строения", case=False, na=False)
                &
                (
                    d["плземучквм"].isna()
                    | (to_num(d["плземучквм"]) <= 0)
                    | (to_num(d["плземучквм"]) < 100)
                )
            )
        },
    }

    # ---------- каскад ----------
    summary, error_blocks = [], []
    cur = df.copy()
    total_initial = len(cur)

    for cond_no, spec in conditions.items():
        mask = spec["mask"](cur)
        err_idx = cur.index[mask]
        errors_df = cur.loc[err_idx].copy()

        total_rows = len(cur)
        error_count = len(errors_df)
        error_percent = (error_count / total_rows * 100) if total_rows else 0

        summary.append({
            "Условие": cond_no,
            "Описание": spec["desc"],
            "Ошибок": int(error_count),
            "Всего строк": int(total_rows),
            "% ошибок": round(error_percent, 2)
        })

        if error_count:
            errors_df["Условие"] = cond_no
            error_blocks.append(errors_df)

        cur = cur.drop(index=err_idx).copy()

    final_rows = len(cur)
    final_percent = round(final_rows / total_initial * 100, 2)
    summary.append({
        "Условие": "✅ ИТОГ",
        "Описание": "Чистых строк после всех проверок",
        "Ошибок": "",
        "Всего строк": final_rows,
        "% ошибок": f"Осталось {final_percent}%"
    })

    return summary, error_blocks, cur


# ---------- обработка всех листов ----------
excel = pd.ExcelFile(INPUT_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:
    for sheet_name in excel.sheet_names:
        print(f"🔍 Обработка листа: {sheet_name}")
        df = excel.parse(sheet_name)

        filial_col = find_column(df, "филиал")
        if filial_col and df[filial_col].notna().any():
            filial_name = str(df[filial_col].dropna().iloc[0])[:31]
        else:
            filial_name = sheet_name[:31]

        summary, errors, clean = process_df(df, filial_name)

        # Пишем summary
        summary_df = pd.DataFrame(summary)
        summary_df.to_excel(writer, sheet_name=filial_name, index=False)
        start_row = len(summary_df) + 3

        # Пишем ошибки
        for block in errors:
            cond_no = int(block["Условие"].iloc[0])
            block.drop(columns=["Условие"]).to_excel(writer, sheet_name=filial_name, index=False, startrow=start_row)
            start_row += len(block) + 3

        # Пишем чистые данные
        clean.to_excel(writer, sheet_name=filial_name, index=False, startrow=start_row + 1)

print(f"✅ Готово! Все филиалы сохранены в {OUTPUT_FILE}")